$$x = y$$

In [38]:
import numpy as np
import pandas as pd

def file_path(name):
    """
    Shortcut function to get the relative path to the directory
    which contains the data.
    """
    return "./data/%s" % name

def country_codes():
    """
    Build country rows from their names, ISO codes, and Numeric
    Country Codes.
    """
    return (
        pd.read_csv(
            file_path(
                "Country_List_ISO_3166_Codes_Latitude_Longitude.csv"),
            usecols=[0, 2, 3],
            index_col=1,
            keep_default_na=False))

def freedom_index():
    """
    Read data from the Freedom Index.
    """
    # TODO: Add xlrd to requirements.
    xl = pd.ExcelFile(file_path("Freedom_index.xlsx"))
    return xl.parse(1)

def ab_values():
    """
    Read generated A/B values for each country.
    """
    return pd.read_excel(file_path("A&B values for RTS.xlsx")).T

def passport_index():
    """
    Read data from the Passport Index.
    """
    return pd.read_excel(file_path("PassportIndex.xlsx"))

def un_stock():
    """
    Read from "Trends in International Migrant Stock: Migrants by
    Destination and Origin"
    """
    un_pd = pd.read_excel(
        file_path(
            "UN_MigrantStockByOriginAndDestination_2015.xlsx"
        ),
        skiprows=15
    )
    un_np = np.array(un_pd)
    ccountry_names = un_np[:,1]
    num_codes = un_np[:,3]
    return un_pd

In [39]:
pd.set_option("display.max_rows", None)

In [40]:
wb_codes = pd.read_csv(file_path("wb-codes.csv"), index_col=0)
ab = ab_values()
ab.index = [wb_codes[wb_codes.index == x]["ISO3"][0] for x in ab_values().index]
ab

,A,B
BDI,72.993,0.0228
NER,113.170,0.0200
MWI,58.581,0.0226
LBR,144.660,0.0197
MOZ,74.334,0.0267
MDG,86.596,0.0252
SLE,155.360,0.0199
GIN,158.650,0.0200
ZAR,94.449,0.0256
CAF,43.254,0.0338


In [41]:
#[cc[cc['Country'] == x]["Alpha-3 code"] if len(cc[cc['Country'] == x]) > 0 else x for x in fi['Country']]
codes = []
other_codes = pd.read_csv(file_path("other.csv"), index_col=0)
cc = country_codes()
fi = freedom_index()
for country in fi['Country']:
    if len(cc[cc['Country'] == country]):
        codes.append(cc[cc['Country'] == country].index[0])
    elif len(other_codes[other_codes.index == country]):
        codes.append(other_codes[other_codes.index == country]["ISO"][0])
    else:
        print(country)
fi['Country'] = codes
fi

,Country,Total Aggr
0,AFG,24
1,ALB,68
2,DZA,35
3,AND,95
4,AGO,24
5,ATG,83
6,ARG,82
7,ARM,45
8,AUS,98
9,AUT,95


In [43]:
pi = passport_index()
#[cc[cc['Country'] == x]["Alpha-3 code"] if len(cc[cc['Country'] == x]) > 0 else x for x in fi['Country']]
codes = []
other_codes = pd.read_csv(file_path("other.csv"), index_col=0)
for country in pi['Country']:
    if len(cc[cc['Country'] == country]):
        codes.append(cc[cc['Country'] == country].index[0])
    elif len(other_codes[other_codes.index == country]):
        codes.append(other_codes[other_codes.index == country]["ISO"][0])
    else:
        print(country)
pi['Country'] = codes
pi

,Country,Rank (1 = most welcoming)
0,KHM,1
1,COM,1
2,CIV,1
3,GNB,1
4,MDG,1
5,MDV,1
6,MRT,1
7,FSM,1
8,MOZ,1
9,WSM,1


In [54]:
passport_index = pi.set_index("Country")
freedom_index = fi.set_index("Country")
#pd.concat([ab, passport_index, freedom_index])
data = ab.join(passport_index).join(freedom_index).dropna()

In [71]:
political_barriers = 2/(1/data['Total Aggr']/sum(1/data['Total Aggr']) +
 data['Rank (1 = most welcoming)']/sum(data['Rank (1 = most welcoming)']))

In [72]:
political_barriers /= sum(political_barriers)
political_barriers

AGO    0.002683
ALB    0.005715
ARG    0.006256
ARM    0.006595
AUS    0.005036
AUT    0.007003
AZE    0.002091
BDI    0.002412
BEL    0.007003
BEN    0.005100
BFA    0.005115
BGD    0.008034
BGR    0.006593
BIH    0.005961
BLR    0.003100
BLZ    0.007808
BOL    0.010757
BRA    0.006481
BTN    0.003773
BWA    0.007009
CAF    0.001617
CAN    0.005250
CHE    0.007028
CHL    0.006979
CHN    0.002145
CIV    0.011293
CMR    0.002725
COG    0.002988
COG    0.002921
COL    0.006033
COM    0.011929
CPV    0.018561
CRI    0.007186
CYP    0.006979
CZE    0.006979
DEU    0.007003
DJI    0.005527
DNK    0.007052
DOM    0.007110
DZA    0.003287
ECU    0.010604
EGY    0.004166
ESP    0.006979
EST    0.006979
ETH    0.001901
FIN    0.007122
FJI    0.006719
FRA    0.006876
FSM    0.019859
GAB    0.003106
GBR    0.006911
GEO    0.006033
GHA    0.005117
GIN    0.003729
GMB    0.003588
GNB    0.008730
GRC    0.006711
GTM    0.005419
GUY    0.004950
HND    0.004930
HRV    0.006796
HTI    0.008120
HUN    0